Загрузим необходимые для работы библиотеки

In [1]:
import pandas as pd
import nltk
import sklearn
import numpy as np

from collections import deque
from io import StringIO

In [2]:
from nltk.corpus import stopwords
import string

In [3]:
#nltk.download()

Считаем данные из файла с обучающей выборкой

In [24]:
train = pd.read_csv("D:/Practice/data/train_first_lab/train.csv", index_col= "id")

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Таким образом, задача сводится к задаче бинарной классификации отзывов на положительные и отрицательные по множеству признаков, представленному словами

In [25]:
train

,text,is_positive
id,,
1,"Sehr gutes Restaurant, leckeres essen und nett...",1
2,Viel Auswahl täuscht über die wahre Tatsache h...,0
3,Hole in the wall burrito joint with the BEST b...,1
4,Pokey Sticks are the best!,1
5,Love their subs. Cheap and top shelf ingredients.,1
6,Best pizza I've ever had. My favorite is the ...,1
7,Good stuff. Pricey by normal pizza standards.,1
8,"New club in the Hard Rock, and currently the h...",1
9,Love their pizza. They used to have a great ta...,1


In [26]:
train = train[0:50000]

Добавим стоп-слова, которые потребуются для задания модели

In [10]:
stop = stopwords.words('english') + list(string.punctuation)

для преобразования коллекции текстовых документов в матрицу подсчета токенов (слов) воспользуемся классом CountVectorizer

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words=stop)
X_train_counts = count_vect.fit_transform(train.text)

Преобразуем полученную матрицу встречаемости слов в документах в нормализованный вид

In [28]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

## Обучение

In [15]:
test = pd.read_csv("D:/Practice/data/train_first_lab/test.csv", index_col="id")

In [16]:

from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline


Чтобы с цепочкой vectorizer => transformer => classifier было проще работать, воспользуемся классом Pipeline, который функционирует как составной (конвейерный) классификатор. Конечным классификатором выступит машина опорных векторов с обучением SGD, которая умеет работать с разреженными матрицами

In [29]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier())
                    ])
text_clf.fit(train.text,train.is_positive)  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

In [30]:
text_clf

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False))])

Теперь, собственно, осталось только предсказать результат классификации на тестовой выборке и записать полученную информацию в файл.

In [31]:
predicted = text_clf.predict(test.text)

In [32]:
predicted

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [33]:
d = {'id': test.index, 'is_positive': np.around(predicted, decimals=0)}
result = pd.DataFrame(data=d)

In [34]:
result.to_csv("D:\\results_lab1_simple.csv", index = False)

Такие результаты получались при тесте на части тренировочной выборки

In [21]:
err_train = np.mean(result.is_positive != test.is_positive)
print(err_train)

0.0955477738869


In [22]:
from sklearn import metrics
fpr, tpr, thresholds = metrics.roc_curve(test.is_positive, predicted)
metrics.auc(fpr, tpr)

0.80911734603197938